### Neural Network Model

THis notebook will contain the Neural Network taht will do the text generation

I will have a FFNN generate job descriptions, will I do so:

1- First part is the tokenization part take the jb description and have the tokens
2- Create my vocabulary:
        - First I will try with every single toke,
        - SecondI will try with only the top N most frequent token
3- For the token representation I will use a Enbeddings, pre-trained embeddings wit h word2Vec
4- How will I train the FFNN ?
   - I will create Input, Output pairs
        - The Input will be a context window of fixed size N and a Special token <START> and the output is the actual next token
        it will be as followed
        | Input (context window, N=3) | Output (next token) |
        |--------------------------------------|---------------------|
        | ["<START>", "Analyze", "data"]       |     "and"           |
    


Perhaps try dropping job descriptions that are over the thrshold to keep the repartition as gaussean bell (Normal law distribuion)




In [2]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..')) # Adjust '..' if your notebook is deeper
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
import nltk

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    print("Downloading NLTK 'punkt' tokenizer...")
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    print("Downloading NLTK 'stopwords'...")
    nltk.download('stopwords')

# Add this check and download for punkt_tab
try:
    # Check for the specific English directory within punkt_tab
    nltk.data.find('tokenizers/punkt_tab/english/') 
except LookupError:
    print("Downloading NLTK 'punkt_tab'...")
    nltk.download('punkt_tab')

print("NLTK resources checked/downloaded.")


NLTK resources checked/downloaded.


In [4]:
import pandas as pd
from models.src.preprocessing.tokenizer import tokenize
from data.process_data_modeling import get_processed_data
from sklearn.model_selection import train_test_split
import os
import logging
import nltk
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import trange

embedding_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

/home/cedric/.cache/pypoetry/virtualenvs/nlp-linkedin-offers-oiOM3zqO-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/cedric/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/cedric/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2025-05-02 09:16:47.007320: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746170207.047568   65226 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746170207.059654   65226 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746170207.145112   65226 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746170207.145121   65226 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746170207.145123   65226 computation_placer.cc:177] computation placer alr

In [5]:


nltk.download('punkt')
nltk.download('stopwords')

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

[nltk_data] Downloading package punkt to /home/cedric/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/cedric/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df = get_processed_data()
df.head()
df.drop(columns=["company_name", "location"], inplace=True)
df.head()


2025-05-02 09:16:50,609 - INFO - Importing data from Kaggle
2025-05-02 09:16:50,610 - INFO - Path to dataset files: /home/cedric/.cache/kagglehub/datasets/arshkon/linkedin-job-postings/versions/13
2025-05-02 09:16:50,611 - INFO - List of files in the dataset: ['companies', 'postings.csv', 'mappings', 'jobs']
2025-05-02 09:16:53,702 - INFO - Rows with at least one NaN value: 1725
2025-05-02 09:16:53,703 - INFO - Number of rows before dropping NaN values: 123849
2025-05-02 09:16:53,724 - INFO - Number of rows after dropping NaN values: 122124
2025-05-02 09:17:35,933 - INFO - DataFrame saved to: /home/cedric/Desktop/MAJEUR/NLP/NLP_Linkedin_offers/models/src/generation/data/processed/cleaned_postings_modeling.parquet
2025-05-02 09:17:35,933 - INFO - Data processing completed successfully !
2025-05-02 09:17:35,934 - INFO - Returning the processed DataFrame


,title,description
0,Marketing Coordinator,job description a leading real estate firm in ...
2,Assitant Restaurant Manager,the national exemplar is accepting application...
3,Senior Elder Law / Trusts and Estates Associat...,senior associate attorney elder law trusts and...
5,Economic Development and Planning Intern,job summary the economic development planning ...
6,Producer,company description raw cereal is a creative d...


In [7]:
df["merged"] = df['title'] + " <START> " + df['description'] + " <END>"
df.drop(columns=["title", "description"], inplace=True)
df.head()


,merged
0,Marketing Coordinator <START> job description ...
2,Assitant Restaurant Manager <START> the nation...
3,Senior Elder Law / Trusts and Estates Associat...
5,Economic Development and Planning Intern <STAR...
6,Producer <START> company description raw cerea...


In [8]:


df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train.head()



,merged
88431,Big Spring Market - Overnight Perishables Rep ...
94806,Front Desk Guest Services Representative <STAR...
67484,AVP Head of National Markets <START> job descr...
29134,Air Export Operations Specialist <START> well ...
24667,Outside Sales Representative - Medical Equipme...


In [9]:
def get_vocab(df:pd.DataFrame) -> dict:
    """
    Get the vocabulary from the dataframe and the size of the vocabulary
    """
    vocab = {}
    for _, row in df.iterrows():
        merged_string = row["merged"]
        tokens = merged_string.split(" ")
        for token in tokens:
            if token not in vocab:
                vocab[token] = len(vocab)
    return vocab, len(vocab)


In [10]:
class FeedForwardNN:
    def __init__(self, df, vocab, input_dim, hidden_dim, output_dim):
        """
        Initialize the FeedForwardNN
        """
        logging.info(f"Initializing FeedForwardNN with input_dim: {input_dim}, hidden_dim: {hidden_dim}, output_dim: {output_dim}")
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.last_window_tokens_index = 0
        self.embedding_transformer = embedding_model
        self.model = self.init_model()
        self.N = 4
        self.vocab = vocab

    def _get_vocab_size(self) -> int:
        """
        Get the size of the vocabulary
        """
        if self.vocab is None:
            raise ValueError("Vocabulary is not initialized")
        return len(self.vocab)

    def _get_context_windows_batched(self, training_set: pd.DataFrame, N: int, batch_size: int):
        """
        Generator that yields batches of (context_window, next_token) pairs.
        """
        logging.info(f"..... Creating context windows .....")
        if len(training_set) < N:
            raise ValueError(f"Window size {N} is larger than the number of tokens {len(training_set)}")
        
        batch_context_windows = []
        for _, row in training_set.iterrows():
            merged_string = row["merged"]
            tokens = merged_string.split(" ")
            for i in range(len(tokens) - N):
                context_window = tokens[i:i+N]
                next_token = tokens[i+N]
                batch_context_windows.append((context_window, next_token))
                if len(batch_context_windows) == batch_size:
                    yield batch_context_windows
                    batch_context_windows = []
        if batch_context_windows:
            yield batch_context_windows

        logging.info(f"..... Finished yielding context windows .....")

            
    def init_model(self):
        """
        Initialize the model
        """
        return nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim),
            nn.ReLU(),
            nn.Linear(self.hidden_dim, self.hidden_dim),
            nn.ReLU(),
            nn.Linear(self.hidden_dim, self.output_dim)
        )

    def _embed_tokens(self, tokens):
        """
        Embed the tokens using the pre-trained embeddings
        """
        sentence_ = " ".join(tokens)
        return self.embedding_transformer.encode(sentence_, batch_size=32, show_progress_bar=True)


    def fit(self, training_set: pd.DataFrame, batch_size=32, epochs=10, device='cpu'):
        """
        Train the FFNN using transformer embeddings as input, with batch context window generation for memory efficiency.
        """
        logging.info(f"..... Training started .....")
        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=1e-3)

        for epoch in range(epochs):
            logging.info(f"Epoch {epoch+1}/{epochs}")
            batch_gen = self._get_context_windows_batched(training_set, self.N, batch_size)
            batch_idx = 0
            for batch in batch_gen:
                context_windows = [" ".join(window) for window, _ in batch]
                next_tokens = [token for _, token in batch]
                y = torch.tensor([self.vocab.get(token, self.vocab.get('<UNK>', 0)) for token in next_tokens], dtype=torch.long, device=device)
                X = self.embedding_transformer.encode(context_windows, batch_size=batch_size, show_progress_bar=False)
                X = torch.tensor(X, dtype=torch.float32, device=device)

                optimizer.zero_grad()
                logits = self.model(X)
                loss = loss_fn(logits, y)
                loss.backward()
                optimizer.step()

                batch_idx += 1
                if batch_idx % 10 == 0:
                    logging.info(f"Epoch {epoch+1}, Batch {batch_idx}: Loss = {loss.item():.4f}")
            logging.info(f"Epoch {epoch+1} completed.")
                



    def _generation_loop(self, context_window, max_length, temperature):
        """
        Generate a job description using the model
        """
        pass
        

In [11]:
embedding_dim = embedding_model.get_sentence_embedding_dimension()
print(embedding_dim)

384


In [12]:
df_train_sample = df_train.sample(n=1000, random_state=42)
vocab, vocab_size = get_vocab(df_train_sample)
network = FeedForwardNN(df_train_sample, vocab, embedding_dim, 1024, vocab_size)
network.fit(df_train_sample, batch_size=8, epochs=1)


2025-05-02 09:17:50,326 - INFO - Initializing FeedForwardNN with input_dim: 384, hidden_dim: 1024, output_dim: 457772
2025-05-02 09:17:51,749 - INFO - ..... Training started .....
2025-05-02 09:17:51,750 - INFO - Epoch 1/1
2025-05-02 09:17:51,750 - INFO - ..... Creating context windows .....


: 